In [1]:
import os
# We'll be dumping and reading the data from this directory
DATA_DIR_PATH = os.path.join(os.getcwd())
CORA_PATH = os.path.join(DATA_DIR_PATH, 'cora')  # this is checked-in no need to make a directory

#
# Cora specific constants
#

# Thomas Kipf et al. first used this split in GCN paper and later Petar Veličković et al. in GAT paper
CORA_TRAIN_RANGE = [0, 140]  # we're using the first 140 nodes as the training nodes
CORA_VAL_RANGE = [140, 140+500]
CORA_TEST_RANGE = [1708, 1708+1000]
CORA_NUM_INPUT_FEATURES = 1433
CORA_NUM_CLASSES = 7

In [2]:
# First let's define these simple functions for loading/saving Pickle files - we need them for Cora

# All Cora data is stored as pickle
def pickle_read(path):
    with open(path, 'rb') as file:
        data = pickle.load(file)

    return data

def pickle_save(path, data):
    with open(path, 'wb') as file:
        pickle.dump(data, file, protocol=pickle.HIGHEST_PROTOCOL)

In [3]:
# We'll pass the training config dictionary a bit later
def load_graph_data(training_config, device):
    dataset_name = training_config['dataset_name'].lower()
    should_visualize = training_config['should_visualize']

    if dataset_name == DatasetType.CORA.name.lower():

        # shape = (N, FIN), where N is the number of nodes and FIN is the number of input features
        node_features_csr = pickle_read(os.path.join(CORA_PATH, 'node_features.csr'))
        # shape = (N, 1)
        node_labels_npy = pickle_read(os.path.join(CORA_PATH, 'node_labels.npy'))
        # shape = (N, number of neighboring nodes) <- this is a dictionary not a matrix!
        adjacency_list_dict = pickle_read(os.path.join(CORA_PATH, 'adjacency_list.dict'))

        # Normalize the features (helps with training)
        node_features_csr = normalize_features_sparse(node_features_csr)
        num_of_nodes = len(node_labels_npy)

        # shape = (2, E), where E is the number of edges, and 2 for source and target nodes. Basically edge index
        # contains tuples of the format S->T, e.g. 0->3 means that node with id 0 points to a node with id 3.
        topology = build_edge_index(adjacency_list_dict, num_of_nodes, add_self_edges=True)

        # Note: topology is just a fancy way of naming the graph structure data 
        # (aside from edge index it could be in the form of an adjacency matrix)

        if should_visualize:  # network analysis and graph drawing
            plot_in_out_degree_distributions(topology, num_of_nodes, dataset_name)  # we'll define these in a second
            visualize_graph(topology, node_labels_npy, dataset_name)

        # Convert to dense PyTorch tensors

        # Needs to be long int type because later functions like PyTorch's index_select expect it
        topology = torch.tensor(topology, dtype=torch.long, device=device)
        node_labels = torch.tensor(node_labels_npy, dtype=torch.long, device=device)  # Cross entropy expects a long int
        node_features = torch.tensor(node_features_csr.todense(), device=device)

        # Indices that help us extract nodes that belong to the train/val and test splits
        train_indices = torch.arange(CORA_TRAIN_RANGE[0], CORA_TRAIN_RANGE[1], dtype=torch.long, device=device)
        val_indices = torch.arange(CORA_VAL_RANGE[0], CORA_VAL_RANGE[1], dtype=torch.long, device=device)
        test_indices = torch.arange(CORA_TEST_RANGE[0], CORA_TEST_RANGE[1], dtype=torch.long, device=device)

        return node_features, node_labels, topology, train_indices, val_indices, test_indices
    else:
        raise Exception(f'{dataset_name} not yet supported.')

In [4]:
def normalize_features_sparse(node_features_sparse):
    assert sp.issparse(node_features_sparse), f'Expected a sparse matrix, got {node_features_sparse}.'

    # Instead of dividing (like in normalize_features_dense()) we do multiplication with inverse sum of features.
    # Modern hardware (GPUs, TPUs, ASICs) is optimized for fast matrix multiplications! ^^ (* >> /)
    # shape = (N, FIN) -> (N, 1), where N number of nodes and FIN number of input features
    node_features_sum = np.array(node_features_sparse.sum(-1))  # sum features for every node feature vector

    # Make an inverse (remember * by 1/x is better (faster) then / by x)
    # shape = (N, 1) -> (N)
    node_features_inv_sum = np.power(node_features_sum, -1).squeeze()

    # Again certain sums will be 0 so 1/0 will give us inf so we replace those by 1 which is a neutral element for mul
    node_features_inv_sum[np.isinf(node_features_inv_sum)] = 1.

    # Create a diagonal matrix whose values on the diagonal come from node_features_inv_sum
    diagonal_inv_features_sum_matrix = sp.diags(node_features_inv_sum)

    # We return the normalized features.
    return diagonal_inv_features_sum_matrix.dot(node_features_sparse)

In [5]:
def build_edge_index(adjacency_list_dict, num_of_nodes, add_self_edges=True):
    source_nodes_ids, target_nodes_ids = [], []
    seen_edges = set()

    for src_node, neighboring_nodes in adjacency_list_dict.items():
        for trg_node in neighboring_nodes:
            # if this edge hasn't been seen so far we add it to the edge index (coalescing - removing duplicates)
            if (src_node, trg_node) not in seen_edges:  # it'd be easy to explicitly remove self-edges (Cora has none..)
                source_nodes_ids.append(src_node)
                target_nodes_ids.append(trg_node)

                seen_edges.add((src_node, trg_node))

    if add_self_edges:
        source_nodes_ids.extend(np.arange(num_of_nodes))
        target_nodes_ids.extend(np.arange(num_of_nodes))

    # shape = (2, E), where E is the number of edges in the graph
    edge_index = np.row_stack((source_nodes_ids, target_nodes_ids))

    return edge_index

In [9]:
import torch
from torch import nn
class GATLayer(torch.nn.Module):
    """
    Implementation #3 was inspired by PyTorch Geometric: https://github.com/rusty1s/pytorch_geometric

    But, it's hopefully much more readable! (and of similar performance)

    """
    
    # We'll use these constants in many functions so just extracting them here as member fields
    src_nodes_dim = 0  # position of source nodes in edge index
    trg_nodes_dim = 1  # position of target nodes in edge index

    # These may change in the inductive setting - leaving it like this for now (not future proof)
    nodes_dim = 0      # node dimension (axis is maybe a more familiar term nodes_dim is the position of "N" in tensor)
    head_dim = 1       # attention head dim

    def __init__(self, num_in_features, num_out_features, num_of_heads, concat=True, activation=nn.ELU(),
                 dropout_prob=0.6, add_skip_connection=True, bias=True, log_attention_weights=False):

        super().__init__()

        self.num_of_heads = num_of_heads
        self.num_out_features = num_out_features
        self.concat = concat  # whether we should concatenate or average the attention heads
        self.add_skip_connection = add_skip_connection

        #
        # Trainable weights: linear projection matrix (denoted as "W" in the paper), attention target/source
        # (denoted as "a" in the paper) and bias (not mentioned in the paper but present in the official GAT repo)
        #

        # You can treat this one matrix as num_of_heads independent W matrices
        self.linear_proj = nn.Linear(num_in_features, num_of_heads * num_out_features, bias=False)

        # After we concatenate target node (node i) and source node (node j) we apply the "additive" scoring function
        # which gives us un-normalized score "e". Here we split the "a" vector - but the semantics remain the same.
        # Basically instead of doing [x, y] (concatenation, x/y are node feature vectors) and dot product with "a"
        # we instead do a dot product between x and "a_left" and y and "a_right" and we sum them up
        self.scoring_fn_target = nn.Parameter(torch.Tensor(1, num_of_heads, num_out_features))
        self.scoring_fn_source = nn.Parameter(torch.Tensor(1, num_of_heads, num_out_features))

        # Bias is definitely not crucial to GAT - feel free to experiment (I pinged the main author, Petar, on this one)
        if bias and concat:
            self.bias = nn.Parameter(torch.Tensor(num_of_heads * num_out_features))
        elif bias and not concat:
            self.bias = nn.Parameter(torch.Tensor(num_out_features))
        else:
            self.register_parameter('bias', None)

        if add_skip_connection:
            self.skip_proj = nn.Linear(num_in_features, num_of_heads * num_out_features, bias=False)
        else:
            self.register_parameter('skip_proj', None)

        #
        # End of trainable weights
        #

        self.leakyReLU = nn.LeakyReLU(0.2)  # using 0.2 as in the paper, no need to expose every setting
        self.activation = activation
        # Probably not the nicest design but I use the same module in 3 locations, before/after features projection
        # and for attention coefficients. Functionality-wise it's the same as using independent modules.
        self.dropout = nn.Dropout(p=dropout_prob)

        self.log_attention_weights = log_attention_weights  # whether we should log the attention weights
        self.attention_weights = None  # for later visualization purposes, I cache the weights here

        self.init_params()
        
    def forward(self, data):
        #
        # Step 1: Linear Projection + regularization
        #

        in_nodes_features, edge_index = data  # unpack data
        num_of_nodes = in_nodes_features.shape[self.nodes_dim]
        assert edge_index.shape[0] == 2, f'Expected edge index with shape=(2,E) got {edge_index.shape}'

        # shape = (N, FIN) where N - number of nodes in the graph, FIN - number of input features per node
        # We apply the dropout to all of the input node features (as mentioned in the paper)
        # Note: for Cora features are already super sparse so it's questionable how much this actually helps
        in_nodes_features = self.dropout(in_nodes_features)

        # shape = (N, FIN) * (FIN, NH*FOUT) -> (N, NH, FOUT) where NH - number of heads, FOUT - num of output features
        # We project the input node features into NH independent output features (one for each attention head)
        nodes_features_proj = self.linear_proj(in_nodes_features).view(-1, self.num_of_heads, self.num_out_features)

        nodes_features_proj = self.dropout(nodes_features_proj)  # in the official GAT imp they did dropout here as well

        #
        # Step 2: Edge attention calculation
        #

        # Apply the scoring function (* represents element-wise (a.k.a. Hadamard) product)
        # shape = (N, NH, FOUT) * (1, NH, FOUT) -> (N, NH, 1) -> (N, NH) because sum squeezes the last dimension
        # Optimization note: torch.sum() is as performant as .sum() in my experiments
        scores_source = (nodes_features_proj * self.scoring_fn_source).sum(dim=-1)
        scores_target = (nodes_features_proj * self.scoring_fn_target).sum(dim=-1)

        # We simply copy (lift) the scores for source/target nodes based on the edge index. Instead of preparing all
        # the possible combinations of scores we just prepare those that will actually be used and those are defined
        # by the edge index.
        # scores shape = (E, NH), nodes_features_proj_lifted shape = (E, NH, FOUT), E - number of edges in the graph
        scores_source_lifted, scores_target_lifted, nodes_features_proj_lifted = self.lift(scores_source, scores_target, nodes_features_proj, edge_index)
        scores_per_edge = self.leakyReLU(scores_source_lifted + scores_target_lifted)

        # shape = (E, NH, 1)
        attentions_per_edge = self.neighborhood_aware_softmax(scores_per_edge, edge_index[self.trg_nodes_dim], num_of_nodes)
        # Add stochasticity to neighborhood aggregation
        attentions_per_edge = self.dropout(attentions_per_edge)

        #
        # Step 3: Neighborhood aggregation
        #

        # Element-wise (aka Hadamard) product. Operator * does the same thing as torch.mul
        # shape = (E, NH, FOUT) * (E, NH, 1) -> (E, NH, FOUT), 1 gets broadcast into FOUT
        nodes_features_proj_lifted_weighted = nodes_features_proj_lifted * attentions_per_edge

        # This part sums up weighted and projected neighborhood feature vectors for every target node
        # shape = (N, NH, FOUT)
        out_nodes_features = self.aggregate_neighbors(nodes_features_proj_lifted_weighted, edge_index, in_nodes_features, num_of_nodes)

        #
        # Step 4: Residual/skip connections, concat and bias
        #

        out_nodes_features = self.skip_concat_bias(attentions_per_edge, in_nodes_features, out_nodes_features)
        return (out_nodes_features, edge_index)

    #
    # Helper functions (without comments there is very little code so don't be scared!)
    #

    def neighborhood_aware_softmax(self, scores_per_edge, trg_index, num_of_nodes):
        """
        As the fn name suggest it does softmax over the neighborhoods. Example: say we have 5 nodes in a graph.
        Two of them 1, 2 are connected to node 3. If we want to calculate the representation for node 3 we should take
        into account feature vectors of 1, 2 and 3 itself. Since we have scores for edges 1-3, 2-3 and 3-3
        in scores_per_edge variable, this function will calculate attention scores like this: 1-3/(1-3+2-3+3-3)
        (where 1-3 is overloaded notation it represents the edge 1-3 and its (exp) score) and similarly for 2-3 and 3-3
         i.e. for this neighborhood we don't care about other edge scores that include nodes 4 and 5.

        Note:
        Subtracting the max value from logits doesn't change the end result but it improves the numerical stability
        and it's a fairly common "trick" used in pretty much every deep learning framework.
        Check out this link for more details:

        https://stats.stackexchange.com/questions/338285/how-does-the-subtraction-of-the-logit-maximum-improve-learning

        """
        # Calculate the numerator. Make logits <= 0 so that e^logit <= 1 (this will improve the numerical stability)
        scores_per_edge = scores_per_edge - scores_per_edge.max()
        exp_scores_per_edge = scores_per_edge.exp()  # softmax

        # Calculate the denominator. shape = (E, NH)
        neigborhood_aware_denominator = self.sum_edge_scores_neighborhood_aware(exp_scores_per_edge, trg_index, num_of_nodes)

        # 1e-16 is theoretically not needed but is only there for numerical stability (avoid div by 0) - due to the
        # possibility of the computer rounding a very small number all the way to 0.
        attentions_per_edge = exp_scores_per_edge / (neigborhood_aware_denominator + 1e-16)

        # shape = (E, NH) -> (E, NH, 1) so that we can do element-wise multiplication with projected node features
        return attentions_per_edge.unsqueeze(-1)

    def sum_edge_scores_neighborhood_aware(self, exp_scores_per_edge, trg_index, num_of_nodes):
        # The shape must be the same as in exp_scores_per_edge (required by scatter_add_) i.e. from E -> (E, NH)
        trg_index_broadcasted = self.explicit_broadcast(trg_index, exp_scores_per_edge)

        # shape = (N, NH), where N is the number of nodes and NH the number of attention heads
        size = list(exp_scores_per_edge.shape)  # convert to list otherwise assignment is not possible
        size[self.nodes_dim] = num_of_nodes
        neighborhood_sums = torch.zeros(size, dtype=exp_scores_per_edge.dtype, device=exp_scores_per_edge.device)

        # position i will contain a sum of exp scores of all the nodes that point to the node i (as dictated by the
        # target index)
        neighborhood_sums.scatter_add_(self.nodes_dim, trg_index_broadcasted, exp_scores_per_edge)

        # Expand again so that we can use it as a softmax denominator. e.g. node i's sum will be copied to
        # all the locations where the source nodes pointed to i (as dictated by the target index)
        # shape = (N, NH) -> (E, NH)
        return neighborhood_sums.index_select(self.nodes_dim, trg_index)

    def aggregate_neighbors(self, nodes_features_proj_lifted_weighted, edge_index, in_nodes_features, num_of_nodes):
        size = list(nodes_features_proj_lifted_weighted.shape)  # convert to list otherwise assignment is not possible
        size[self.nodes_dim] = num_of_nodes  # shape = (N, NH, FOUT)
        out_nodes_features = torch.zeros(size, dtype=in_nodes_features.dtype, device=in_nodes_features.device)

        # shape = (E) -> (E, NH, FOUT)
        trg_index_broadcasted = self.explicit_broadcast(edge_index[self.trg_nodes_dim], nodes_features_proj_lifted_weighted)
        # aggregation step - we accumulate projected, weighted node features for all the attention heads
        # shape = (E, NH, FOUT) -> (N, NH, FOUT)
        out_nodes_features.scatter_add_(self.nodes_dim, trg_index_broadcasted, nodes_features_proj_lifted_weighted)

        return out_nodes_features

    def lift(self, scores_source, scores_target, nodes_features_matrix_proj, edge_index):
        """
        Lifts i.e. duplicates certain vectors depending on the edge index.
        One of the tensor dims goes from N -> E (that's where the "lift" comes from).

        """
        src_nodes_index = edge_index[self.src_nodes_dim]
        trg_nodes_index = edge_index[self.trg_nodes_dim]

        # Using index_select is faster than "normal" indexing (scores_source[src_nodes_index]) in PyTorch!
        scores_source = scores_source.index_select(self.nodes_dim, src_nodes_index)
        scores_target = scores_target.index_select(self.nodes_dim, trg_nodes_index)
        nodes_features_matrix_proj_lifted = nodes_features_matrix_proj.index_select(self.nodes_dim, src_nodes_index)

        return scores_source, scores_target, nodes_features_matrix_proj_lifted

    def explicit_broadcast(self, this, other):
        # Append singleton dimensions until this.dim() == other.dim()
        for _ in range(this.dim(), other.dim()):
            this = this.unsqueeze(-1)

        # Explicitly expand so that shapes are the same
        return this.expand_as(other)

    def init_params(self):
        """
        The reason we're using Glorot (aka Xavier uniform) initialization is because it's a default TF initialization:
            https://stackoverflow.com/questions/37350131/what-is-the-default-variable-initializer-in-tensorflow

        The original repo was developed in TensorFlow (TF) and they used the default initialization.
        Feel free to experiment - there may be better initializations depending on your problem.

        """
        nn.init.xavier_uniform_(self.linear_proj.weight)
        nn.init.xavier_uniform_(self.scoring_fn_target)
        nn.init.xavier_uniform_(self.scoring_fn_source)

        if self.bias is not None:
            torch.nn.init.zeros_(self.bias)

    def skip_concat_bias(self, attention_coefficients, in_nodes_features, out_nodes_features):
        if self.log_attention_weights:  # potentially log for later visualization in playground.py
            self.attention_weights = attention_coefficients

        if self.add_skip_connection:  # add skip or residual connection
            if out_nodes_features.shape[-1] == in_nodes_features.shape[-1]:  # if FIN == FOUT
                # unsqueeze does this: (N, FIN) -> (N, 1, FIN), out features are (N, NH, FOUT) so 1 gets broadcast to NH
                # thus we're basically copying input vectors NH times and adding to processed vectors
                out_nodes_features += in_nodes_features.unsqueeze(1)
            else:
                # FIN != FOUT so we need to project input feature vectors into dimension that can be added to output
                # feature vectors. skip_proj adds lots of additional capacity which may cause overfitting.
                out_nodes_features += self.skip_proj(in_nodes_features).view(-1, self.num_of_heads, self.num_out_features)

        if self.concat:
            # shape = (N, NH, FOUT) -> (N, NH*FOUT)
            out_nodes_features = out_nodes_features.view(-1, self.num_of_heads * self.num_out_features)
        else:
            # shape = (N, NH, FOUT) -> (N, FOUT)
            out_nodes_features = out_nodes_features.mean(dim=self.head_dim)

        if self.bias is not None:
            out_nodes_features += self.bias

        return out_nodes_features if self.activation is None else self.activation(out_nodes_features)

In [12]:
import torch.nn as nn
from torch.optim import Adam
import numpy as np

class GAT(torch.nn.Module):
    """
    The most interesting and hardest implementation is implementation #3.
    Imp1 and imp2 differ in subtle details but are basically the same thing.

    So I'll focus on imp #3 in this notebook.

    """

    def __init__(self, num_of_layers, num_heads_per_layer, num_features_per_layer, add_skip_connection=True, bias=True,
                 dropout=0.6, log_attention_weights=False):
        super().__init__()
        assert num_of_layers == len(num_heads_per_layer) == len(num_features_per_layer) - 1, f'Enter valid arch params.'

        num_heads_per_layer = [1] + num_heads_per_layer  # trick - so that I can nicely create GAT layers below

        gat_layers = []  # collect GAT layers
        for i in range(num_of_layers):
            layer = GATLayer(
                num_in_features=num_features_per_layer[i] * num_heads_per_layer[i],  # consequence of concatenation
                num_out_features=num_features_per_layer[i+1],
                num_of_heads=num_heads_per_layer[i+1],
                concat=True if i < num_of_layers - 1 else False,  # last GAT layer does mean avg, the others do concat
                activation=nn.ELU() if i < num_of_layers - 1 else None,  # last layer just outputs raw scores
                dropout_prob=dropout,
                add_skip_connection=add_skip_connection,
                bias=bias,
                log_attention_weights=log_attention_weights
            )
            gat_layers.append(layer)

        self.gat_net = nn.Sequential(
            *gat_layers,
        )

    # data is just a (in_nodes_features, edge_index) tuple, I had to do it like this because of the nn.Sequential:
    # https://discuss.pytorch.org/t/forward-takes-2-positional-arguments-but-3-were-given-for-nn-sqeuential-with-linear-layers/65698
    def forward(self, data):
        return self.gat_net(data)

In [1]:
import enum
import pickle
import scipy.sparse as sp

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # checking whether you have a GPU

class DatasetType(enum.Enum):
    CORA = 0
    
config = {
    'dataset_name': DatasetType.CORA.name,
    'should_visualize': False
}

node_features, node_labels, edge_index, train_indices, val_indices, test_indices = load_graph_data(config, device)

print(node_features.shape, node_features.dtype)
print(node_labels.shape, node_labels.dtype)
print(edge_index.shape, edge_index.dtype)
print(train_indices.shape, train_indices.dtype)
print(val_indices.shape, val_indices.dtype)
print(test_indices.shape, test_indices.dtype)

/home/dwalke/miniconda3/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


NameError: name 'torch' is not defined

In [14]:
import pandas as pd
pd.DataFrame(node_features[0].cpu().numpy()).describe()

,0
count,1433.000000
mean,0.000698
std,0.008781
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,0.111111


In [15]:
node_features[train_indices].device

device(type='cuda', index=0)

In [1]:
from xgboost import XGBClassifier
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.utils import add_self_loops

dataset = Planetoid(root='/tmp/Cora', name='Cora', split="public") ## public
dataset.transform = T.NormalizeFeatures()
data = dataset[0]

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

X = data.x.to(device) #node_features # dataset[0].x.to(device) #node_features # dataset[0].x.to(device)
y = data.y.to(device) #node_labels #dataset[0].y.to(device)

test = data.test_mask.to(device) #test_indices #dataset[0].test_mask.to(device)
train = data.train_mask.to(device) #train_indices # dataset[0].train_mask.to(device)
val = data.val_mask.to(device) #val_indices # dataset[0].val_mask.to(device)


y_train = y[train].to(device)
y_val = y[val].to(device)
y_test = y[test].to(device)

edge_index = data.edge_index.to(device) #edge_index#dataset[0].edge_index.to(device)
edge_index = add_self_loops(edge_index)[0]
print(train.shape)
print(val.shape)
print(test.shape)
train

/home/dwalke/miniconda3/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


torch.Size([2708])
torch.Size([2708])
torch.Size([2708])


tensor([ True,  True,  True,  ..., False, False, False], device='cuda:0')

In [2]:
edge_index

tensor([[   0,    0,    0,  ..., 2705, 2706, 2707],
        [ 633, 1862, 2582,  ..., 2705, 2706, 2707]], device='cuda:0')

In [3]:
from torch_geometric.nn import GATConv
from torch.nn import ELU, Softmax, Dropout
from torch import nn
import numpy as np
import torch.nn.functional as F

def glorot(shape):
    """Glorot & Bengio (AISTATS 2010) init."""
    init_range = np.sqrt(6.0/(shape[0]+shape[1]))
    initial_tensor = torch.FloatTensor(shape[0], shape[1]).uniform_(-init_range, init_range).type(torch.float32)
    return torch.nn.parameter.Parameter(data=initial_tensor, requires_grad=True)

##glorot & bengio init; early stoppping
class GAT(nn.Module):
    def __init__(self, hidden_dim, out_dim):
        super(GAT, self).__init__()
        heads = 8
        self.conv1 = GATConv(X.shape[1], hidden_dim, heads=heads, dropout=0.6, add_self_loops=False)
        self.conv2 = GATConv(hidden_dim*heads, out_dim, dropout=0.6, heads=1, concat=False, add_self_loops=False)
        
    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()
        
    def forward(self, data):
        X, edge_index = data
        p = 0.6
        X = F.dropout(X, p=p, training=self.training)
        X, (edges, attn_weights_1) = self.conv1(X, edge_index, return_attention_weights=True)
        X = F.elu(X)
        X = F.dropout(X, p=p, training=self.training)
        X, (edges, attn_weights_2) = self.conv2(X, edge_index, return_attention_weights=True)
        return F.log_softmax(X, dim=1), (attn_weights_1, attn_weights_2)

In [17]:
from torch.optim import Adam, SGD
import torch
from torch import tensor
import copy

# def evaluate(model, mask):
#     global X, edge_index, y_test
    
#     with torch.inference_mode():
#         model.eval()
#         logits, (attn_weights_1, attn_weights_2) = model((X,edge_index))
#         y_pred = logits.argmax(-1)
#         y_pred_test = y_pred[mask]
#         matching = (y_pred_test == y_test).sum()
#         acc = matching / y_test.shape[0]
#     return acc
accs = []
accuracy = None
best_model = None
for i in range(1):
    
    model = GAT(8, 7).to(device)# GAT(2, [8,1], [X.shape[1], 8, 7], add_skip_connection=False, bias=True,
                     #dropout=0.6, log_attention_weights=True).to(device) #GAT(8, dataset.num_classes).to(device)
    model.reset_parameters()
    optim = Adam(model.parameters(), lr=0.005, weight_decay=0.0005 )
    loss_fn = nn.CrossEntropyLoss(reduction='mean')
    last_loss = 0
    # increased_loss = 0
    patience = 100

    min_loss = 0
    max_acc = 0
    curr_step  = 0

    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    epochs = 1000

    for epoch in range(epochs):
        if epoch % 100 == 0:
            print(f"Epoch: {str(epoch)}") 
        model.train()
        logits, (attn_weights_1, attn_weights_2) = model((X,edge_index))
        
    #     loss = loss_fn(logits[train], y_train)
        loss = F.nll_loss(logits[train], y_train)
        optim.zero_grad()
        loss.backward()
        optim.step()

        train_losses.append(loss.item())

        model.eval()
        with torch.inference_mode():
            logits,(attn_weights_1, attn_weights_2) = model((X,edge_index))
            val_loss = F.nll_loss(logits[val], y_val)
            y_pred = logits.argmax(-1)
            y_pred_val = y_pred[val]
            matching = (y_pred_val == y_val).sum()
            acc = matching / y_val.shape[0]


#             acc = acc.cpu().numpy()

#             if acc >= max_acc or loss.item() <= min_loss:
#                 curr_step = 0
#                 max_acc = max(acc, max_acc)
#                 min_loss = min(loss.item(), min_loss)
#             else:
#                 curr_step += 1
#                 if patience == curr_step:
#                     print(epoch)
#                     break
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            with torch.inference_mode():
                model.eval()
                logits, (attn_weights_1, attn_weights_2) = model((X,edge_index))
                y_pred = logits.argmax(-1)
                y_pred_test = y_pred[test]
                matching = (y_pred_test == y_test).sum()
                accuracy = matching / y_test.shape[0]
                best_model = copy.deepcopy(model)

        val_losses.append(val_loss.item())
        if patience > 0 and epoch > epochs // 2:
            tmp = tensor(val_losses[-(patience + 1):-1])
            if val_loss > tmp.mean().item():
                break
    print(accuracy)
    
    accs.append(accuracy)

Epoch: 0
Epoch: 100
Epoch: 200
Epoch: 300
Epoch: 400
Epoch: 500
tensor(0.8340, device='cuda:0')


In [22]:
from sklearn.metrics import accuracy_score
import pandas as pd

df = pd.DataFrame()

with torch.inference_mode():
    model.eval()
    logits, (attn_weights_1, attn_weights_2) = best_model((X,edge_index))
    y_pred = logits.argmax(-1)
    y_pred_test = y_pred[test].cpu()
    df["pred"] = y_pred_test.cpu()
    df["true"] = y_test.cpu()
df.to_csv("gat_pred_0834.csv")

In [143]:
logits, (attn_weights_1, attn_weights_2) = model((X,edge_index))

In [138]:
## If features of neighboring nodes are similar to each other (e.g. based on cosine similarity), how high are the respective attention weights

In [139]:
attn_weights_1[:,1]

tensor([0.2983, 0.2400, 0.2829,  ..., 0.4678, 0.2193, 0.1894], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [140]:
from scipy import stats

source_lift = X.index_select(0, edge_index[0])
target_lift = X.index_select(0, edge_index[1])

cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
score = cos(source_lift, target_lift)
exp_score = torch.exp(score)
summed_exp_score = torch.zeros_like(exp_score).scatter(0, edge_index[1],exp_score, reduce="add")
target_lifted_summed_exp_score = summed_exp_score.index_select(0, edge_index[1])
normalized_scores = exp_score / target_lifted_summed_exp_score
y = normalized_scores.cpu().numpy()
print(y.shape)
for i in range(8):
    x = attn_weights_1[:,i].squeeze().cpu().detach().numpy()
    print(x.shape)
    res = stats.pearsonr(x, y)
    print(res)
    
x = attn_weights_2.squeeze().cpu().detach().numpy()
res = stats.pearsonr(x, y)
print(res)

(13264,)
(13264,)
PearsonRResult(statistic=0.7705918116989339, pvalue=0.0)
(13264,)
PearsonRResult(statistic=0.7524490012402268, pvalue=0.0)
(13264,)
PearsonRResult(statistic=0.7774745653128273, pvalue=0.0)
(13264,)
PearsonRResult(statistic=0.7601042068634979, pvalue=0.0)
(13264,)
PearsonRResult(statistic=0.7808525261111766, pvalue=0.0)
(13264,)
PearsonRResult(statistic=0.7802990223019417, pvalue=0.0)
(13264,)
PearsonRResult(statistic=0.7772642331363361, pvalue=0.0)
(13264,)
PearsonRResult(statistic=0.7771160567103308, pvalue=0.0)
PearsonRResult(statistic=0.7814432715763204, pvalue=0.0)


In [46]:
model.state_dict()

OrderedDict([('conv1.att_src',
              tensor([[[-0.2039,  0.4802, -0.4523,  0.2791,  0.1785,  0.2704,  0.1466,
                        -0.1215],
                       [-0.2673,  0.4094,  0.0263,  0.4597,  0.3036,  0.3103, -0.1237,
                        -0.0064],
                       [ 0.3324, -0.3757, -0.1585,  0.2742, -0.2757, -0.4746, -0.1436,
                        -0.4312],
                       [ 0.4503,  0.1946, -0.2227, -0.3955, -0.0531, -0.2799, -0.4734,
                        -0.0487],
                       [ 0.0528,  0.4646, -0.2926, -0.0858,  0.3943, -0.0182, -0.3293,
                        -0.2251],
                       [ 0.5097, -0.0889, -0.1569, -0.3378, -0.4267,  0.1275, -0.4548,
                        -0.0540],
                       [ 0.2209,  0.1460, -0.2964,  0.5937, -0.3204, -0.0217, -0.2116,
                         0.4255],
                       [ 0.1549, -0.3991, -0.1879,  0.4243, -0.0795,  0.3225, -0.0078,
                        -0.1255]]],

In [27]:
model.gat_net[0].attention_weights[:,0].shape#[0].squeeze()

AttributeError: 'GAT' object has no attribute 'gat_net'

In [28]:
from scipy import stats
for i in range(8):
    print(stats.pearsonr(model.gat_net[0].attention_weights[:, i].squeeze().cpu(), normalized_scores.cpu()))

AttributeError: 'GAT' object has no attribute 'gat_net'

In [ ]:
from scipy import stats
stats.pearsonr(model.gat_net[1].attention_weights.squeeze().cpu(), normalized_scores.cpu()) ##todo other layers

In [ ]:
model.gat_net[1].attention_weights.squeeze()

In [ ]:
normalized_scores.cpu()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(epoch+1), train_losses)
plt.plot(range(epoch+1), val_losses)
plt.legend(["train", "val"])